In [2]:
import pandas as pd
import numpy as np
import datetime
import warnings
# warnings.simplefilter('ignore')

from sklearn.ensemble import GradientBoostingRegressor

In [17]:
flights = pd.read_excel('Flights.xlsx')
pilots = pd.read_excel('Pilots.xlsx')

In [6]:
flights.head()

,pilot_id,aircraft_id,date,weather,time,duration,sex,age,p9,p10,p11,p12,p13,p14
0,46,1,2017-01-01,1,0,276,0,26,68,68,54,62,62,65
1,46,1,2017-01-02,0,0,268,0,26,67,67,53,66,61,72
2,5,0,2017-01-03,1,1,141,0,29,51,61,63,61,53,47
3,6,0,2017-01-04,0,0,390,1,32,66,68,51,57,66,66
4,46,0,2017-01-05,0,0,486,0,26,67,68,53,64,64,70


In [10]:
weathers = {'Хорошие': 0, 'Плохие': 1}
daytimes = {'Светлое': 0, 'Темное': 1}

In [65]:
weather, daytime = 'Хорошие', 'Темное'
aircraft_id = 1
duration = 220
threshold = 65

In [21]:
weather_id, daytime_id = weathers[weather], daytimes[daytime]

In [13]:
X_train = flights[['aircraft_id', 'weather', 'time', 'duration', 'sex', 'age', 'p9', 'p10', 'p11']]
y_train = flights[['p12', 'p13', 'p14']].mean(axis=1)

In [16]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [19]:
pilots.head(1)

,sex,bd,p9,p10,p11,p12,p13,p14,trend
0,1,1981-04-27,57,61,67,50,56,59,-1


In [47]:
curdate = datetime.date(2018, 5, 13)

X_test = pd.DataFrame(np.zeros((pilots.shape[0], X_train.shape[1])),
                      columns=X_train.columns,
                      index=pilots.index
                     )

X_test['aircraft_id'] = aircraft_id
X_test['weather'] = weather_id
X_test['time'] = daytime_id
X_test['duration'] = duration
X_test['sex'] = pilots['sex']
X_test['age'] = (curdate - pilots['bd'].apply(lambda x: x.date())).apply(lambda x: x.days) // 365

for pilot_id in X_test.index:
    last_values = flights[flights['pilot_id'] == pilot_id][['p9', 'p10', 'p11']].iloc[-1, :]
    X_test.loc[pilot_id, ['p9', 'p10', 'p11']] = last_values

In [51]:
predicted = model.predict(X_test)

In [85]:
result = pd.DataFrame(columns=['predicted'], index=X_test.index)
result['predicted'] = predicted
result = result.sort_values(by='predicted', ascending=False)
result = result.iloc[:7, :]
#result = result[result['predicted'] >= threshold]
# result['predicted'] = result['predicted'].apply(lambda x: round(x, 1))
# result = result.rename(columns={'predicted': 'Оценка'})

In [86]:
text = ('Id пилота   Оценка\n'
        + '\n'.join(['{:9d}   {:.1f}'.format(i, result.at[i, 'predicted']) for i in result.index])
       )
text

'Id пилота   Оценка\n       27   70.5\n       43   68.2\n        2   66.6\n       29   66.0\n       15   65.8\n       26   65.4\n       40   64.6'

In [57]:
[list(predicted)]

[[61.11494675768362,
  61.87905178360277,
  66.58540346297342,
  62.20732827790907,
  59.16354486971209,
  58.81615574205828,
  62.90715787010016,
  61.23072998448651,
  60.73920537622944,
  59.00233258801993,
  60.37481475008326,
  60.692132342108664,
  56.05793128459735,
  61.33173697413665,
  59.04919835277485,
  65.82489644235447,
  63.33152087643316,
  62.656591900029184,
  60.78755676931326,
  61.77751153148936,
  61.856740882766665,
  59.49738139240476,
  62.58323144477373,
  61.32218225324652,
  58.37947078081612,
  62.031867348827674,
  65.43400142006612,
  70.54868334583138,
  62.92906430301059,
  66.00839030670505,
  60.78717092746211,
  61.737589484477,
  63.51051557742105,
  62.57588271909896,
  57.06104533720628,
  59.61324592823011,
  62.10674624188487,
  63.45886205065683,
  62.529769748248434,
  63.18741655359883,
  64.63095367273591,
  62.72082715439143,
  62.62834396589186,
  68.22536145457192,
  56.04639596425744,
  62.65445390060209,
  60.94273924839565,
  64.36435

In [52]:
predicted

array([61.11494676, 61.87905178, 66.58540346, 62.20732828, 59.16354487,
       58.81615574, 62.90715787, 61.23072998, 60.73920538, 59.00233259,
       60.37481475, 60.69213234, 56.05793128, 61.33173697, 59.04919835,
       65.82489644, 63.33152088, 62.6565919 , 60.78755677, 61.77751153,
       61.85674088, 59.49738139, 62.58323144, 61.32218225, 58.37947078,
       62.03186735, 65.43400142, 70.54868335, 62.9290643 , 66.00839031,
       60.78717093, 61.73758948, 63.51051558, 62.57588272, 57.06104534,
       59.61324593, 62.10674624, 63.45886205, 62.52976975, 63.18741655,
       64.63095367, 62.72082715, 62.62834397, 68.22536145, 56.04639596,
       62.6544539 , 60.94273925, 64.36435923, 60.51135278, 60.67672752])

In [49]:
X_test.head()

,aircraft_id,weather,time,duration,sex,age,p9,p10,p11
0,1,0,1,220,1,37,57.0,62.0,65.0
1,1,0,1,220,1,55,60.0,57.0,64.0
2,1,0,1,220,1,24,50.0,48.0,68.0
3,1,0,1,220,1,53,54.0,60.0,54.0
4,1,0,1,220,0,47,55.0,48.0,67.0


In [50]:
pilot_id = 3
flights[flights['pilot_id'] == pilot_id]

,pilot_id,aircraft_id,date,weather,time,duration,sex,age,p9,p10,p11,p12,p13,p14
14,3,1,2017-01-15,1,0,114,1,51,53,61,55,60,72,66
24,3,0,2017-01-25,0,1,409,1,51,54,61,55,52,71,64
42,3,0,2017-02-12,1,1,508,1,52,53,62,55,57,71,63
201,3,0,2017-07-21,0,1,249,1,52,52,60,55,51,65,59
276,3,0,2017-10-04,0,0,333,1,52,54,60,56,59,76,64
296,3,1,2017-10-24,1,0,556,1,52,54,61,53,56,68,61
304,3,0,2017-11-01,0,0,354,1,52,54,60,54,61,72,67


In [37]:
(curdate - pilots.at[1, 'bd'].date()).days // 365

55

In [77]:
from PIL import ImageTk, Image

In [76]:
!pip install PIL

  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
